# Concept  
팩토리얼 (Factorial)   
개념 : 자연수 1 ~ n 까지의 모든 정수 곱  
표기법 : n!  
참고 : 순열과 조합 등 대부분의 계산의 기초.   
사용처 : 숫자의 곱셈 계산, 경우의 수 계산  

# Implementing  


In [ ]:
# 설계
# (1) n -> n-1 -> n-2 ... 1 혹은 1 -> 2 -> ... -> n 순으로 반복된다.
# (2) 재귀함수를 사용한다.

In [46]:
# 재귀함수수
# 최초 작성
def factorial_custom_recursion(n, result=None):
    # check input value
    if isinstance(n, int) != True:
        print('input value is not int type. please check again')
    elif n < 0:
        print('input number is under 0. please check again.')
    else:
        pass
    # factorial algorithm
    if result == None:
        result = n
        return factorial_custom_recursion(n-1, result)
    else:
        result = result * (n)
        if n-1 == 1:
            return result
        else:
            return factorial_custom_recursion(n-1, result)
        
# 리팩토링
def factorial_custom_recursion(n):
    # check input value
    if isinstance(n, int) != True:
        raise ValueError('Input value is not int type.')
    if n < 0:
        raise ValueError('Input value must be a positive integer.')
    # factorial algorythm
    if (n == 0) or (n == 1):
        return 1
    else:
        return n * factorial_custom_recursion(n-1)

In [47]:
def factorial_custom_for(n):
    # check input value
    if isinstance(n, int) != True:
        raise ValueError('Input value is not int type.')
    if n < 0:
        raise ValueError('Input value must be a positive integer.')
    # factorial algorythm
    result = 1
    for i in range(1, n+1):
        result = result * i
    return result

# Library

파이썬의 내장 모듈인 math.factorial 함수는 C로 구현된 CPython 표준 라이브러리에 의해 제공됩니다. 이 함수는 CPython의 소스 코드에 정의되어 있습니다.

```c
static PyObject *
math_factorial(PyObject *module, PyObject *arg)
/*[clinic end generated code: output=6686f26fae00e9ca input=713fb771677e8c31]*/
{
    long x, two_valuation;
    int overflow;
    PyObject *result, *odd_part;

    // Convert input value from python long type to C long type
    x = PyLong_AsLongAndOverflow(arg, &overflow);
    if (x == -1 && PyErr_Occurred()) {  // Check Exception
        return NULL;
    }
    else if (overflow == 1) {  // Check if the argument is under C's Max Long value
        PyErr_Format(PyExc_OverflowError,
                     "factorial() argument should not exceed %ld",
                     LONG_MAX);
        return NULL;
    }
    else if (overflow == -1 || x < 0) {  // Check if the argument is negative integer
        PyErr_SetString(PyExc_ValueError,
                        "factorial() not defined for negative values");
        return NULL;
    }

    /* use lookup table if x is small */ // --> 하단에 추가함함
    if (x < (long)Py_ARRAY_LENGTH(SmallFactorials))
        return PyLong_FromUnsignedLong(SmallFactorials[x]);

    /* else express in the form odd_part * 2**two_valuation, and compute as
       odd_part << two_valuation. */
    odd_part = factorial_odd_part(x);
    if (odd_part == NULL)
        return NULL;
    two_valuation = x - count_set_bits(x);
    result = _PyLong_Lshift(odd_part, two_valuation);
    Py_DECREF(odd_part);
    return result;
}
```

```c
/* Lookup table for small factorial values */

static const unsigned long SmallFactorials[] = {
    1, 1, 2, 6, 24, 120, 720, 5040, 40320,
    362880, 3628800, 39916800, 479001600,
#if SIZEOF_LONG >= 8
    6227020800, 87178291200, 1307674368000,
    20922789888000, 355687428096000, 6402373705728000,
    121645100408832000, 2432902008176640000
#endif
};
```

```c
static PyObject *
factorial_odd_part(unsigned long n) // 홀수만 계산
{
    long i;
    unsigned long v, lower, upper;
    PyObject *partial, *tmp, *inner, *outer;

    inner = PyLong_FromLong(1);
    if (inner == NULL)
        return NULL;
    outer = Py_NewRef(inner);

    upper = 3;
    for (i = _Py_bit_length(n) - 2; i >= 0; i--) {
        v = n >> i;
        if (v <= 2)
            continue;
        lower = upper;
        /* (v + 1) | 1 = least odd integer strictly larger than n / 2**i */
        upper = (v + 1) | 1;
        /* Here inner is the product of all odd integers j in the range (0,
           n/2**(i+1)].  The factorial_partial_product call below gives the
           product of all odd integers j in the range (n/2**(i+1), n/2**i]. */
        partial = factorial_partial_product(lower, upper, _Py_bit_length(upper-2));
        /* inner *= partial */
        if (partial == NULL)
            goto error;
        tmp = PyNumber_Multiply(inner, partial);
        Py_DECREF(partial);
        if (tmp == NULL)
            goto error;
        Py_SETREF(inner, tmp);
        /* Now inner is the product of all odd integers j in the range (0,
           n/2**i], giving the inner product in the formula above. */

        /* outer *= inner; */
        tmp = PyNumber_Multiply(outer, inner);
        if (tmp == NULL)
            goto error;
        Py_SETREF(outer, tmp);
    }
    Py_DECREF(inner);
    return outer;

  error:
    Py_DECREF(outer);
    Py_DECREF(inner);
    return NULL;
}
```

```c
static unsigned long
count_set_bits(unsigned long n)
{
    unsigned long count = 0;
    while (n != 0) {
        ++count;
        n &= n - 1; /* clear least significant bit */
    }
    return count;
}
```

# Speed Check

In [56]:
# Using Library
import math
import time

test_candits = range(1, 1000)

start = time.time()
for i in range(0, 100):
    for test_case in test_candits:
        result = math.factorial(test_case)
end = time.time()
print(f'총 계산 시간 : {end - start}')
print(f'1회 당 계산 시간 : {(end - start)/100}')
print(f'마지막 값 : {result}')

총 계산 시간 : 1.552440881729126
1회 당 계산 시간 : 0.015524408817291259
마지막 값 : 402387260077093773543702433923003985719374864210714632543799910429938512398629020592044208486969404800479988610197196058631666872994808558901323829669944590997424504087073759918823627727188732519779505950995276120874975462497043601418278094646496291056393887437886487337119181045825783647849977012476632889835955735432513185323958463075557409114262417474349347553428646576611667797396668820291207379143853719588249808126867838374559731746136085379534524221586593201928090878297308431392844403281231558611036976801357304216168747609675871348312025478589320767169132448426236131412508780208000261683151027341827977704784635868170164365024153691398281264810213092761244896359928705114964975419909342221566832572080821333186116811553615836546984046708975602900950537616475847728421889679646244945160765353408198901385442487984959953319101723355556602139450399736280750137837615307127761926849034352625200015888535147331611702103968175

In [66]:
# Using Custom Def - recursion
import time

test_candits = range(1, 1000)

start = time.time()
for i in range(0, 100):
    for test_case in test_candits:
        result = factorial_custom_recursion(test_case)
end = time.time()
print(f'총 계산 시간 : {end - start}')
print(f'1회 당 계산 시간 : {(end - start)/100}')
print(f'마지막 값 : {result}')

총 계산 시간 : 22.75756072998047
1회 당 계산 시간 : 0.2275756072998047
마지막 값 : 40238726007709377354370243392300398571937486421071463254379991042993851239862902059204420848696940480047998861019719605863166687299480855890132382966994459099742450408707375991882362772718873251977950595099527612087497546249704360141827809464649629105639388743788648733711918104582578364784997701247663288983595573543251318532395846307555740911426241747434934755342864657661166779739666882029120737914385371958824980812686783837455973174613608537953452422158659320192809087829730843139284440328123155861103697680135730421616874760967587134831202547858932076716913244842623613141250878020800026168315102734182797770478463586817016436502415369139828126481021309276124489635992870511496497541990934222156683257208082133318611681155361583654698404670897560290095053761647584772842188967964624494516076535340819890138544248798495995331910172335555660213945039973628075013783761530712776192684903435262520001588853514733161170210396817592

In [58]:
# Using Custom Def - for
import time

test_candits = range(1, 1000)

start = time.time()
for i in range(0, 100):
    for test_case in test_candits:
        result = factorial_custom_for(test_case)
end = time.time()
print(f'총 계산 시간 : {end - start}')
print(f'1회 당 계산 시간 : {(end - start)/100}')
print(f'마지막 값 : {result}')

총 계산 시간 : 10.968559741973877
1회 당 계산 시간 : 0.10968559741973877
마지막 값 : 402387260077093773543702433923003985719374864210714632543799910429938512398629020592044208486969404800479988610197196058631666872994808558901323829669944590997424504087073759918823627727188732519779505950995276120874975462497043601418278094646496291056393887437886487337119181045825783647849977012476632889835955735432513185323958463075557409114262417474349347553428646576611667797396668820291207379143853719588249808126867838374559731746136085379534524221586593201928090878297308431392844403281231558611036976801357304216168747609675871348312025478589320767169132448426236131412508780208000261683151027341827977704784635868170164365024153691398281264810213092761244896359928705114964975419909342221566832572080821333186116811553615836546984046708975602900950537616475847728421889679646244945160765353408198901385442487984959953319101723355556602139450399736280750137837615307127761926849034352625200015888535147331611702103968175

|계산 방법|횟수|총 계산 시간|1회 당 계산 시간|
|---|---|---|---|
|Library - math.factorial|100|1.552|0.015|
|직접 구축 - 재귀|100|22.955|0.229|
|직접 구축 - 반복문|100|10.969|0.109|

**소수점 4째자리 이하 버림**

# Extracting technical skills  

In [60]:
# (1) 다중 재귀 제한
# 파이썬은 기본적으로 재귀 호출의 깊이를 제한한다.
# 따라서 다중 재귀가 될 가능성이 있는 함수는 재귀가 아닌 다른 방식으로 처리해야 한다.
# 재귀 호출 깊이를 확인하려면 아래 참고고
import sys
print(sys.getrecursionlimit())

3000


In [ ]:
# (2) 분할 정복 (Divide and Conquer)
# 분할 정복이란, 문제를 더 작은 하위 문제들로 나눠 각각을 해결한 후, 이를 합쳐 원래 문제를 해결하는 전략
# 팩토리얼 n! 을 작은 범위의 곱셈으로 분할한 후, 각각을 계산해 최종 결과를 얻는다.
# 이렇게 하면 병렬적 계산이나 재귀적 계산이 가능해진다.

def factorial_divide_and_conquer(start, end):
    if start > end:
        return 1
    if start == end:
        return start
    mid = (start + end) // 2
    return factorial_divide_and_conquer(start, mid) * factorial_divide_and_conquer(mid + 1, end)
    
def factorial_custom(n):
    return factorial_divide_and_conquer(1, n)

test_candits = range(1, 1000)

start = time.time()
for i in range(0, 100):
    for test_case in test_candits:
        result = factorial_custom(test_case)
end = time.time()
print(f'총 계산 시간 : {end - start}')
print(f'1회 당 계산 시간 : {(end - start)/100}')
print(f'마지막 값 : {result}')

총 계산 시간 : 20.114295959472656
1회 당 계산 시간 : 0.20114295959472656
마지막 값 : 402387260077093773543702433923003985719374864210714632543799910429938512398629020592044208486969404800479988610197196058631666872994808558901323829669944590997424504087073759918823627727188732519779505950995276120874975462497043601418278094646496291056393887437886487337119181045825783647849977012476632889835955735432513185323958463075557409114262417474349347553428646576611667797396668820291207379143853719588249808126867838374559731746136085379534524221586593201928090878297308431392844403281231558611036976801357304216168747609675871348312025478589320767169132448426236131412508780208000261683151027341827977704784635868170164365024153691398281264810213092761244896359928705114964975419909342221566832572080821333186116811553615836546984046708975602900950537616475847728421889679646244945160765353408198901385442487984959953319101723355556602139450399736280750137837615307127761926849034352625200015888535147331611702103968175

In [ ]:
# (3) 홀짝 나누기 + 비트 연산 활용

# (3-1) 홀짝 나누기
# 홀짝을 나누는 이유는  계산 속도의 효율성을 위해서이다.
# n!의 짝수 부분의 경우 2^짝수개수 로 계산을 대체할 수 있다. (3! 의 짝수 부분 : 2 / 4!의 짝수 부분 2 * 4 = 8 = 2^3 / 6!의 짝수 부분 : 48 ..?) ===> ???

# (3-2) 비트 연산
# 비트 연산은 곱셈이나 나눗셈보다 연산 속도가 빠르다.
# 비트 연산의 예시

In [70]:
count = 0
n = 1
count += n & 1
count += n & 1
count += n & 1
count

3

In [73]:
n = 8
n >>= 1
n

4

## Reference  

[chatGPT](https://chatgpt.com/share/677b5229-5ac8-800f-966c-0d019f95d404)  
[source code of Cpython math.factorial](https://github.com/python/cpython/blob/main/Modules/mathmodule.c#L1986)  
[docs of python - math.factorial](https://github.com/python/cpython/blob/main/Modules/mathmodule.c#L1986)  